In [1]:
import numpy as np
from numpy import zeros, ones, eye
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
#logical inputs with biases in the last column
X = tf.constant([[0,0,1],[0,1,1],[1,0,1],[1,1,1]], dtype=tf.float32)

AND = tf.constant([[0],[0],[0],[1]], dtype=tf.float32) # also tf.float32 implicitly
OR = tf.constant([[0],[1],[1],[1]], dtype=tf.float32) # also tf.float32 implicitly
NOR = tf.constant([[1],[0],[0],[0]], dtype=tf.float32) # also tf.float32 implicitly
NAND = tf.constant([[1],[1],[1],[0]], dtype=tf.float32) # also tf.float32 implicitly
XOR = tf.constant([[0],[1],[1],[0]], dtype=tf.float32) # also tf.float32 implicitly
RHO = tf.constant([[.0001],[.001],[.01],[.01],[.1],[1]])
# squares = map_fn(lambda x: x * x, elems)
# J = map_fn(lambda y, t: 1/2*np.mean((y-t)**2) )
J = lambda y, t: 1/2*np.mean((y-t)**2)

In [3]:
# sess.run(INPUTS)
w = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32)
y_bar = x @ w #tensorflow operation

w_k_plus_1 = 0
sess = tf.Session()

In [4]:
# initializing w_k, w_k_plus_1
w_k = np.array([0,0,0])
# w_k_plus_1 = sess.run({w_k_plus_1: [[0],[0],[0]]})
len(sess.run(RHO))

print(sess.run(X) @ w_k)

[0. 0. 0. 0.]


This next part is still broken (perceptron - too many for loops)

In [52]:
epochs = np.array([1000,10_000])
# for AND gate

no_of_iterations_to_converge = []

# for j in range(len(sess.run(RHO))):
rho =  sess.run(RHO)[1]

for i in range(epochs[1]):
    for k in range(len(sess.run(X)[0])):
        X_k  = sess.run(X)[k,:]
        y_b = X_k *w_k
        y_k =  sess.run(NOR)[k,:]
        
        w_k_plus_1 = w_k  + rho*(y_k - y_b) * X_k
        
        w_k = w_k_plus_1
    no_of_iterations_to_converge.append(i)

In [50]:
# w_k
# no_of_iterations_to_converge
w_k

array([1.        , 1.        , 0.66700011])

In [51]:
sess.run(X) @ w_k

array([0.66700011, 1.66700011, 1.66700011, 2.66700011])

In [47]:
 sess.run(X)

array([[0., 0., 1.],
       [0., 1., 1.],
       [1., 0., 1.],
       [1., 1., 1.]], dtype=float32)